### 活动类别识别

In [ ]:
from torchvision import transforms

from utils.RFID_Dataset import RFID_Dataset
from utils.DatasetUtils import DatasetUtils

data_utils= DatasetUtils()

transform = transforms.Compose([
])

train_maps={
    r"data\RFID\final\静止001.csv":0,
    r"data\RFID\final\蹲下001.csv":1,
    r"data\RFID\final\行走001.csv":2,
}

eval_maps={
    r"data\RFID\final\静止002.csv":0,
    r"data\RFID\final\蹲下002.csv":1,
    r"data\RFID\final\行走002.csv":2,
}

# 加载数据集
train_dataset = RFID_Dataset(
    file_label_map=train_maps,
    T=32,
    step=2,
    transform=transform,
)
eval_dataset = RFID_Dataset(
    file_label_map=eval_maps,
    T=32,
    step=2,
    transform=transform,
)

print(f"训练集的数据个数: {len(train_dataset)}")
print(f"验证集的数据个数: {len(eval_dataset)}")

In [ ]:
# 模型组网
from torchkeras import summary
from model.ClassifyNet import ClassifyNet

from utils.ModelWorker.ClassifyModelWorker import ClassifyModelWorker

input_shape=data_utils.get_data_shape(train_dataset)
model=ClassifyNet(
    input_shape=(1,32,12),
    num_classes=3
)

model_worker=ClassifyModelWorker(model)

print(f"{input_shape=}")
model_info=summary(model,input_shape=input_shape)

In [3]:
# 模型准备
from torch.utils.data import DataLoader
from torch import nn
from torch import optim

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=4)

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [4]:
# 模型训练

model_worker.train(
    criterion=loss,
    optimizer=optimizer,
    train_loader=train_loader,
    eval_loader=eval_loader,
    epochs=20,
    enable_board=True,
)

Model will be trained on cuda
Epoch [1/20] begin...


Training: 100%|██████████| 52/52 [00:00<00:00, 75.32step/s, acc=1, loss=0.0265]   


Train Loss: 0.6522, Train Acc: 0.6064


Evaluating: 100%|██████████| 103/103 [00:00<00:00, 469.30step/s, acc=1, loss=0.00168]


Eval samples:409, Eval Loss: 0.3395, Eval Acc: 1.0000
Epoch [1/20] finish
Epoch [2/20] begin...


...

Eval samples:409, Eval Loss: 0.0000, Eval Acc: 1.0000
Epoch [19/20] finish
Epoch [20/20] begin...


Training: 100%|██████████| 52/52 [00:00<00:00, 149.86step/s, acc=1, loss=0.000139]


Train Loss: 0.0000, Train Acc: 1.0000


Evaluating: 100%|██████████| 103/103 [00:00<00:00, 543.70step/s, acc=1, loss=0]      

Eval samples:409, Eval Loss: 0.0000, Eval Acc: 1.0000
Epoch [20/20] finish


In [5]:
# 模型评估

model_worker.evaluate(
    eval_loader=eval_loader,
    criterion=loss,
)

Evaluating: 100%|██████████| 103/103 [00:00<00:00, 179.02step/s, acc=1, loss=0]      

Eval samples:409, Eval Loss: 0.0000, Eval Acc: 1.0000


(4.3373290288896056e-05, 1.0)

In [6]:
# 保存模型
model_worker.save('./output/ACR.pth')

In [ ]:
# 加载模型
model_worker.load('./output/ACR.pth')

In [15]:
# 模型预测

indexs,inputs,labels = data_utils.select_simple(eval_dataset,count=4)
preds,acc = model_worker.predict(inputs=inputs,labels=labels)

print(f"{inputs.shape=}")
print(f"{acc=}")
print(f"indexs:{indexs}")
print(f"labels:{labels}")
print(f"preds :{preds}")

inputs.shape=torch.Size([4, 1, 32, 12])
acc=1.0
indexs:tensor([146, 282, 212,  88])
labels:tensor([1, 2, 1, 0])
preds :tensor([1, 2, 1, 0], device='cuda:0')
